<a href="https://colab.research.google.com/github/proffranciscofernando/house_price_prediction/blob/main/main_pt_BR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão de Preços de Casas na Califórnia - Pipeline de ML

Este notebook demonstra um pipeline de aprendizado de máquina construído com Scikit-Learn para prever os preços das casas na Califórnia. Ele abrange o pré-processamento de dados, treinamento do modelo, otimização de hiperparâmetros usando Grid Search com validação cruzada e uma interface interativa para fazer previsões com base na entrada do usuário.

## 1. Importando Bibliotecas Necessárias

Primeiramente, importamos todas as bibliotecas necessárias e suprimimos quaisquer avisos não críticos para uma saída mais limpa.

In [1]:
# Importando Bibliotecas Necessárias e Suprimindo Avisos

import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib  # Para salvar e carregar o modelo

# Suprimindo avisos
import warnings
warnings.filterwarnings('ignore')

## 2. Carregando e Explorando o Conjunto de Dados

Usaremos o **California Housing Dataset** para prever os preços das casas.

### 2.1 Carregando o Conjunto de Dados

In [2]:
# Carregando o conjunto de dados
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing(as_frame=True)
df = data.frame

# Visualizando as primeiras linhas
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


### 2.2 Análise Exploratória de Dados

Vamos verificar as informações básicas do conjunto de dados para entender sua estrutura.

In [3]:
# Informações básicas sobre o conjunto de dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


### 2.3 Selecionando Recursos e Variável Alvo

Selecionaremos algumas colunas relevantes para simplificar o exemplo.

In [4]:
# Selecionando recursos e variável alvo
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

### 2.4 Tratando Valores Ausentes

Verificaremos se existem valores ausentes nos dados selecionados.

In [5]:
# Verificando valores ausentes
X.isnull().sum()

,0
MedInc,0
HouseAge,0
AveRooms,0
AveBedrms,0
Population,0
AveOccup,0
Latitude,0
Longitude,0


Como podemos ver, não há valores ausentes no conjunto de dados.

## 3. Preparação dos Dados

### 3.1 Dividindo os Dados em Conjuntos de Treinamento e Teste

Dividimos o conjunto de dados em conjuntos de treinamento e teste para avaliar o desempenho dos modelos.

In [6]:
# Dividindo os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 4. Construindo o Pipeline

### 4.1 Criando o Pipeline de Pré-processamento

Definimos transformações para colunas numéricas e categóricas.

In [7]:
# Definindo colunas numéricas (não há colunas categóricas neste conjunto de dados)
numeric_features = X.columns.tolist()
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Como não temos recursos categóricos, não definimos um transformer categórico
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])

## 5. Otimização de Hiperparâmetros com Grid Search

### 5.1 Definindo Modelos e Parâmetros

Definiremos uma lista de modelos de regressão e os hiperparâmetros que desejamos otimizar para cada um.

In [8]:
# Definindo modelos e seus hiperparâmetros
models_params = {
    'Regressão Linear': {
        'model': LinearRegression(),
        'params': {}
    },
    'Ridge Regression': {
        'model': Ridge(),
        'params': {
            'classifier__alpha': [0.1, 1.0, 10.0]
        }
    },
    'Lasso Regression': {
        'model': Lasso(),
        'params': {
            'classifier__alpha': [0.001, 0.01, 0.1, 1.0]
        }
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__max_depth': [None, 5, 10]
        }
    },
    'SVR': {
        'model': SVR(),
        'params': {
            'classifier__C': [0.1, 1, 10],
            'classifier__kernel': ['linear', 'rbf']
        }
    }
}

### 5.2 Executando Grid Search com Validação Cruzada

Agora, realizaremos o Grid Search com validação cruzada para cada modelo.

In [9]:
# Executando Grid Search com validação cruzada para cada modelo
results = []

for name, mp in models_params.items():
    model = mp['model']
    params = mp.get('params', {})

    # Criando o pipeline com o modelo atual
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    if params:
        # Grid Search com validação cruzada
        grid_search = GridSearchCV(
            estimator=pipeline,
            param_grid=params,
            cv=5,
            scoring='neg_mean_squared_error',
            n_jobs=-1
        )

        # Treinando o modelo
        grid_search.fit(X_train, y_train)

        # Melhor modelo encontrado
        best_model = grid_search.best_estimator_
    else:
        # Se não há hiperparâmetros para ajustar
        pipeline.fit(X_train, y_train)
        best_model = pipeline
        grid_search = None

    # Fazendo previsões no conjunto de teste
    y_pred = best_model.predict(X_test)

    # Calculando métricas
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Armazenando os resultados
    results.append({
        'Modelo': name,
        'Melhores Hiperparâmetros': grid_search.best_params_ if params else 'N/A',
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'Pipeline': best_model  # Armazenando o melhor modelo
    })

### 5.3 Comparando Modelos Otimizados

Vamos visualizar as métricas de cada modelo otimizado em um dataframe.

In [10]:
# Comparando os modelos otimizados
df_results = pd.DataFrame(results)
df_results = df_results.sort_values(by='RMSE')
df_results.reset_index(drop=True, inplace=True)
df_results[['Modelo', 'MAE', 'RMSE', 'R²', 'Melhores Hiperparâmetros']]

,Modelo,MAE,RMSE,R²,Melhores Hiperparâmetros
0,Random Forest,0.326995,0.504246,0.805966,"{'classifier__max_depth': None, 'classifier__n..."
1,SVR,0.377445,0.568944,0.752980,"{'classifier__C': 10, 'classifier__kernel': 'r..."
2,Lasso Regression,0.533145,0.744642,0.576856,{'classifier__alpha': 0.001}
3,Ridge Regression,0.533199,0.745579,0.575791,{'classifier__alpha': 0.1}
4,Regressão Linear,0.533200,0.745581,0.575788,N/A


## 6. Selecionando e Salvando o Melhor Modelo

### 6.1 Selecionando o Melhor Modelo

In [11]:
# Selecionando o melhor modelo com base no RMSE
best_model_name = df_results.loc[0, 'Modelo']
best_pipeline = df_results.loc[0, 'Pipeline']

print(f"O melhor modelo foi: {best_model_name}")
print(f"Com hiperparâmetros: {df_results.loc[0, 'Melhores Hiperparâmetros']}")

O melhor modelo foi: Random Forest
Com hiperparâmetros: {'classifier__max_depth': None, 'classifier__n_estimators': 200}


### 6.2 Salvando o Modelo

In [12]:
# Salvando o modelo usando joblib
joblib.dump(best_pipeline, 'best_model.pkl')

['best_model.pkl']

## 7. Carregando o Modelo e Fazendo Previsões

### 7.1 Fazendo Previsões com Novas Entradas do Usuário

Nesta seção, criaremos uma interface iterativa onde o usuário pode inserir novos dados, e o modelo fará previsões com base nesses dados.

In [13]:
# Carregando o modelo salvo
loaded_model = joblib.load('best_model.pkl')

#### Função para Obter Dados de Entrada do Usuário

In [14]:
# Função para obter dados de entrada do usuário
def get_user_data():
    print("Por favor, insira os detalhes da casa:")

    # Solicitando entrada do usuário
    MedInc = input("MedInc (Renda mediana em dezenas de milhares): ")
    while True:
        try:
            MedInc = float(MedInc)
            if MedInc < 0:
                raise ValueError
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número positivo.")
            MedInc = input("MedInc: ")

    HouseAge = input("HouseAge (Idade média das casas na área): ")
    while True:
        try:
            HouseAge = float(HouseAge)
            if HouseAge < 0:
                raise ValueError
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número positivo.")
            HouseAge = input("HouseAge: ")

    AveRooms = input("AveRooms (Média de quartos por casa): ")
    while True:
        try:
            AveRooms = float(AveRooms)
            if AveRooms <= 0:
                raise ValueError
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número positivo.")
            AveRooms = input("AveRooms: ")

    AveBedrms = input("AveBedrms (Média de quartos por casa): ")
    while True:
        try:
            AveBedrms = float(AveBedrms)
            if AveBedrms <= 0:
                raise ValueError
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número positivo.")
            AveBedrms = input("AveBedrms: ")

    Population = input("Population (População do quarteirão): ")
    while True:
        try:
            Population = float(Population)
            if Population <= 0:
                raise ValueError
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número positivo.")
            Population = input("Population: ")

    AveOccup = input("AveOccup (Média de ocupação por casa): ")
    while True:
        try:
            AveOccup = float(AveOccup)
            if AveOccup <= 0:
                raise ValueError
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número positivo.")
            AveOccup = input("AveOccup: ")

    Latitude = input("Latitude: ")
    while True:
        try:
            Latitude = float(Latitude)
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número.")
            Latitude = input("Latitude: ")

    Longitude = input("Longitude: ")
    while True:
        try:
            Longitude = float(Longitude)
            break
        except ValueError:
            print("Valor inválido. Por favor, insira um número.")
            Longitude = input("Longitude: ")

    # Criando um DataFrame com os dados inseridos
    new_house = pd.DataFrame({
        'MedInc': [MedInc],
        'HouseAge': [HouseAge],
        'AveRooms': [AveRooms],
        'AveBedrms': [AveBedrms],
        'Population': [Population],
        'AveOccup': [AveOccup],
        'Latitude': [Latitude],
        'Longitude': [Longitude]
    })

    return new_house

#### Loop Iterativo para Previsões

In [16]:
# Loop iterativo para fazer previsões
while True:
    # Obter dados de entrada do usuário
    new_house = get_user_data()

    # Fazer previsão
    prediction = loaded_model.predict(new_house)

    predicted_price = prediction[0]

    print(f"\nPreço previsto da casa: ${predicted_price * 100000:.2f}\n")

    # Perguntar se o usuário deseja inserir outra casa
    continue_input = input("Gostaria de inserir outra casa? (s/n): ").lower()
    if continue_input != 's':
        print("Encerrando previsões.")
        break

Por favor, insira os detalhes da casa:
MedInc (Renda mediana em dezenas de milhares): 12
HouseAge (Idade média das casas na área): 30
AveRooms (Média de quartos por casa): 3
AveBedrms (Média de quartos por casa): 2
Population (População do quarteirão): 20000
AveOccup (Média de ocupação por casa): 3
Latitude: 2
Longitude: 2

Preço previsto da casa: $385463.84

Gostaria de inserir outra casa? (s/n): n
Encerrando previsões.


## 8. Conclusão

Neste notebook, construímos um pipeline de aprendizado de máquina que inclui múltiplos modelos de regressão. Utilizamos o Grid Search com validação cruzada para otimizar os hiperparâmetros de cada modelo. Avaliamos cada modelo usando métricas como MAE, RMSE e R². Com base nessas métricas, selecionamos o melhor modelo otimizado e o salvamos para uso futuro. Finalmente, implementamos uma interface interativa que permite aos usuários inserir novos dados e obter previsões do modelo, tornando a aplicação prática e interativa.

Esse processo é essencial em projetos de aprendizado de máquina para garantir que estamos selecionando o modelo e os hiperparâmetros mais apropriados para o nosso problema, além de facilitar a implantação e o uso contínuo do modelo em ambientes do mundo real.

## 9. Referências

- [Documentação do Scikit-Learn sobre Pipelines](https://scikit-learn.org/stable/modules/compose.html#pipeline)
- [Documentação do Scikit-Learn sobre GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
- [Salvando Modelos com Joblib](https://scikit-learn.org/stable/modules/model_persistence.html)
- [California Housing Dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset)
- [Validação Cruzada no Scikit-Learn](https://scikit-learn.org/stable/modules/cross_validation.html)
- [Métricas de Regressão](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)